In [136]:
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
import pickle
import pandas as pd
import collections
from nltk.corpus import words

In [2]:
folk_model_1  = models.load_model('all_folk_model_weights/weights-improvement-01-5.6964.hdf5')
folk_model_10 = models.load_model('all_folk_model_weights/weights-improvement-10-4.4159.hdf5')
folk_model_25 = models.load_model('all_folk_model_weights/weights-improvement-25-4.0081.hdf5')
folk_model_89 = models.load_model('all_folk_model_weights/weights-improvement-89-3.6083.hdf5')

In [151]:
def input_cleaner(text_in):
    text_in = text_in.lower().replace(' n ', ' eol ').replace('[verse ', '[verse')
    text_in = text_in.replace("'", '').replace('-', ' ')
    tokens = text_in.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    return tokens



def generate_seed_text_from_input():
    
    input_text = input('Type what you want here.')
    
    print('Rendering text as lyrics...')
    
    seed_text = input_cleaner(input_text)
    
    seed_text = [word_to_index[word] if word in word_to_index else word_to_index['unknown'] for word in seed_text]
    
    seed_text = pad_sequences([seed_text], maxlen=20, value=2500)
    
    return seed_text, input_text



def generate_song_from_input(model, seq_length, reverse_dictionary, n_words, seed_text):
    
    print('Writing new song')
    
    result = []
    
    seed_text = np.squeeze(seed_text, axis=0)
    
    for _ in range(n_words):
        in_text = np.expand_dims(seed_text, axis=0)
#         yhat = model.predict_classes(in_text)
        predict_prob = model.predict_proba(in_text)
        yhat = np.random.choice(range(10001), 1, p=predict_prob[0])
        if yhat == 10000:
            word = words.words()[np.random.randint(0,len(words.words()))]
        else:
            word = reverse_dictionary[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-seq_length:]
        result.append(word)
           
    return ' '.join(result)

def generate_new_song(model):
    seq_length = 20
    reverse_dictionary = index_to_word
    n_words = int(input('How many words?'))
    
    seed_text, input_text = generate_seed_text_from_input()
    
    lyrics = generate_song_from_input(model, seq_length, reverse_dictionary, n_words, seed_text)
    
    lyrics = lyrics.replace(' eol ', ' \n ').replace(' eos ', ' \n\n\n ')
    
    complete_song = input_text + '\n' + lyrics
    
    print('New song finished!')
    
    return complete_song



def generate_song_from_text_seed(model, seq_length, reverse_dictionary, n_words, seed_text):
    
    print('Writing new song')
    
    result = []
    
    for _ in range(n_words):
        in_text = np.expand_dims(seed_text, axis=0)
#         yhat = model.predict_classes(in_text)
        predict_prob = model.predict_proba(in_text)
        yhat = np.random.choice(range(10001), 1, p=predict_prob[0])
        if yhat == 10000:
            word = words.words()[np.random.randint(0,len(words.words()))]
        else:
            word = reverse_dictionary[yhat[0]]
        seed_text = np.append(seed_text, yhat)
        seed_text = seed_text[-seq_length:]
        result.append(word)
           
    return ' '.join(result)



def generate_new_song_from_text_seed(model):

    seq_length = 20
    reverse_dictionary = index_to_word
    n_words = 500
    seed_text = np.array([  138,    42, 10000,     5, 10000,     0,  3519,     3,    13,
           5,     1,   910,     9,   555,    64, 10000,    16,     1,
         742,     9])
    
    input_text = [index_to_word[word] for word in seed_text]
    
    input_text = ' '.join(input_text)
    
    lyrics = generate_song_from_text_seed(model, seq_length, reverse_dictionary, n_words, seed_text)
    
    lyrics = lyrics.replace('eol ', ' \n ').replace(' eol', ' \n ')
    
    lyrics = lyrics.replace(' eol ', ' \n ').replace(' eos ', ' \n\n\n ')
    
    complete_song = lyrics
    
    print('New song finished!')
    
    return input_text, complete_song

In [ ]:
models = [folk_model_1, folk_model_10, folk_model_25, folk_model_89]

In [138]:
index_to_word[10000]

'unknown'

In [154]:
pickle.load(open('model_checkpoints/weights-improvement-01-5.8760.hdf5', 'rb'))

UnpicklingError: invalid load key, 'H'.

In [7]:
word_to_index = pickle.load(open('all_folk_word_to_index.pkl', 'rb'))

index_to_word = pickle.load(open('all_folk_index_to_word.pkl', 'rb'))

In [5]:
seed_text = ['It', 'was', 'a', 'white', 'crane', 'eos', 'It', 'was', 'a', 'helpless', 'thing']

In [10]:
seed_text = np.array([  91,   76,  289,   96,   96,   50, 3139,    0,   43,    4, 1470,
                   10,   74,    5, 2113, 3139,    0,   36,   34, 4894])

In [134]:
in_seed, song = generate_new_song_from_text_seed(folk_model_89)

Writing new song
New song finished!


In [135]:
print(in_seed)
print(song)

him from unknown to unknown eol onward and on to the ends of reason where unknown is the means of
the right 
 oh where seen is hesitation 
 when they smell your prayed tree 
 find it all by your side 
 see things is made it all my mind 
 swim to you get out for name 
 i’m at all the time youve heard 


 help you wake up row see in my head 
 leave you too come back back bring off i take so low 
 its pretty tryin to come home 
 until we were just always the last 


 they were stoned to a wise day 
 that lately im seventeen 
 i will always feel times deep down to black brigade 
 crying inside truth brother do not open too much old love too 
 just a fact of enjoyableness you were an stomachfully to the beat out of rhyme 
 well ninety water a song in white ladies midnight 
 it was more young land 
 it was mistle like they will stand in 
 you gonna first up i looked to do 
 but it seemed is a gastaldo while i figure up 
 verse4 
 soffit cross the rear green 


 give me its all alone 
 im bli

In [150]:
print(generate_new_song(folk_model_89))

How many words?500
Type what you want here.swag
Rendering text as lyrics...
Writing new song
New song finished!
swag
eol 
 eol i i dont dont dont why why i wont no when i 
 is all one thing i can feel the hell so leave 


 please let ol house and bone 
 watched mud 
 jimmy end nice up now 
 admission 


 he said the devil came across the garbage through the rain 
 all the old heliolatrous undine dancing but with she has probably more around 
 he hates to think an outlet for you 
 and all of us grabbing of 
 like lovers 
 our oath 
 just pool shines 
 a shadow of the shadow will be one scene deep as a road 
 children all ever lost the ugly 
 i see beautiful this year 
 still quiet on his heartbeat names 
 ran on it grounds 
 my hands said with a shiny stop 
 but now once  silly pays to be freed 
 and i could questions youll make magic easier i dont want to ever leave you sober and we didn’t care so cast the clubs who are born 
 refrain 
 oh in the healing life 
 in its side of the danci

In [152]:
print(generate_new_song(folk_model_89))

How many words?500
Type what you want here.To my dear, dearest shayma, I love you
Rendering text as lyrics...
Writing new song
New song finished!
To my dear, dearest shayma, I love you
displace solo of carissa 
 ridding out of the bathtub out and like my heart in your time its a bride dream 
 im yours i left loves me its god to be mean 
 but its here i dont know 
 that they wont know 
 its not the word they you told 
 i know youd like it to me 
 i cant hear you with you 
 verse1 
 there are nothing wrong just done was gone 
 dont know how i feel a star 
 check into heaven sleeping 
 i stood in the road all cities to his feet 
 breathing hands clinging to the leaves and easy 
 and god piled aint through gold 
 in the summer missing the tall skin of mind 
 my hearts not caught up 
 you can rise the river close i sink 
 hook 
 now ill be old 
 i dont think of me for you 
 verse2 
 and the summer wind seemed 


 now claim i get out 
 hes made not up 
 i been seeing my teeth 
 the hard face

In [160]:
len(index_to_word)

10000

In [159]:
np.random.choice(range(10000), 20)

array([6177, 6520, 7721,  563, 3529, 3225, 5970, 3530, 8796, 1600, 8312,
       1850, 1457, 5649, 8785, 8441, 6035, 1996,  951, 9471])